# Vault PoV - Database Engine - MariaDB Dynamic Credentials

## Business Value of Capability

We use dynamic secrets to allow applications to get dynamic creds to access databases.### Setup

## Setup

In [1]:
export VAULT_ADDR=http://127.0.0.1:8200
export VAULT_TOKEN=root
export VAULT_SKIP_VERIFY=true
export DB_HOST=192.168.17.167 #or localhost

: 1

### Start Vault Enterprise Server in Dev Mode

In [2]:
docker run --rm -d \
    --name vault \
    -p 8200:8200 \
    -e 'VAULT_DEV_ROOT_TOKEN_ID=root' \
    -e "VAULT_ADDR=http://127.0.0.1:8200" \
    hashicorp/vault-enterprise

5286225dfe15985c0d98441ffa1ebbe3a824dd7722d63ca16caafe4c1649305a


: 1

In [3]:
vault login root

WARNING! The VAULT_TOKEN environment variable is set! This takes precedence
over the value set by this command. To use the value set by this command,
unset the VAULT_TOKEN environment variable or set it to the token displayed
below.

Success! You are now authenticated. The token information displayed below
is already stored in the token helper. You do NOT need to run "vault login"
again. Future Vault requests will automatically use this token.

Key                  Value
---                  -----
token                root
token_accessor       q0rCjTJwv1Oem4s2Rdj7VZ5M
token_duration       ∞
token_renewable      false
token_policies       ["root"]
identity_policies    []
policies             ["root"]


: 1

In [4]:
vault status

Key             Value
---             -----
Seal Type       shamir
Initialized     true
Sealed          false
Total Shares    1
Threshold       1
Version         1.7.2+ent
Storage Type    inmem
Cluster Name    vault-cluster-f06cc205
Cluster ID      66fe259c-305b-9b5d-dccb-46fa86d3bb6b
HA Enabled      false


: 1

In [ ]:
vault version

### Start the database

In [8]:
# docker start mariadb
docker run --rm -itd \
  --name=mariadb \
  -e PUID=1000 \
  -e PGID=1000 \
  -e MYSQL_ROOT_PASSWORD=password \
  -e TZ=America/New_York \
  -e MYSQL_DATABASE=vaultdemodb `#optional` \
  -e MYSQL_PASSWORD=password `#optional` \
  -p 3306:3306 \
  linuxserver/mariadb

b051792164ef75b7b37d405611e308b863735ad1c0ca0d795b6c21a3a80f40e7


: 1

Create db user for Vault.

In [5]:
docker exec -it mariadb /bin/bash -c "
mysql -u root -ppassword << EOF
GRANT ALL PRIVILEGES ON *.* TO 'vaultadmin'@'%' IDENTIFIED BY 'vadmin' WITH GRANT OPTION;
FLUSH PRIVILEGES;
EOF"

: 1

## Demo
---

**Display existing database users**

In [6]:
docker exec -it mariadb mysql -u vaultadmin -p'vadmin' -e "select user from mysql.user;"

+-------------+
| User        |
+-------------+
| root        |
| vaultadmin  |
| mariadb.sys |
+-------------+


: 1

### Enable Vault database secrets engine

In [7]:
vault secrets enable -path=mdb1 database

Success! Enabled the database secrets engine at: mdb1/


: 1

Confirm secrets engine has been enabled.

In [8]:
vault secrets list

Path          Type         Accessor              Description
----          ----         --------              -----------
cubbyhole/    cubbyhole    cubbyhole_386ad115    per-token private secret storage
identity/     identity     identity_8e33e8ff     identity store
mdb1/         database     database_9c7dbff6     n/a
secret/       kv           kv_d10f81c8           key/value secret storage
sys/          system       system_b9d93945       system endpoints used for control, policy and debugging


: 1

### Configure the Database Secrets Engine

**Configure MySQL secrets for the databases secrets engine**

In [9]:
vault write mdb1/config/mysql \
    plugin_name=mysql-legacy-database-plugin \
    connection_url="{{username}}:{{password}}@tcp(${DB_HOST}:3306)/" \
    allowed_roles="readonly","readonly-long","readonly-short" \
    username="vaultadmin" \
    password="vadmin"

: 1

NOTE: The username and password are templated in the "`connection_url`" string. They get their values from the "`username`" and "`password`" fields.  We do this so that reading the path "mdb1/config/mysql" will not show them.

(optional) Verify that you cannot see the password.

In [10]:
#optional verification
vault read mdb1/config/mysql

Key                                   Value
---                                   -----
allowed_roles                         [readonly readonly-long readonly-short]
connection_details                    map[connection_url:{{username}}:{{password}}@tcp(192.168.17.167:3306)/ username:vaultadmin]
password_policy                       n/a
plugin_name                           mysql-legacy-database-plugin
root_credentials_rotate_statements    []


: 1

### (optional) Rotate root credentials

We can make the configuration of the database secrets engine even more secure by rotating the root credentials (actually just the password) that we passed into the configuration.  We do this by running this command:

In [ ]:
#vault write -force mdb1/rotate-root/mysql

### Create DB roles

**Create a Vault role so applications can access credentials**. Lease of 1 hour. Renewable for 24 hours.

In [11]:
vault write mdb1/roles/readonly-long \
    db_name=mysql \
    creation_statements="CREATE USER '{{name}}'@'%' IDENTIFIED BY '{{password}}';GRANT SELECT ON *.* TO '{{name}}'@'%';" \
    default_ttl="1h" \
    max_ttl="24h"

Success! Data written to: mdb1/roles/readonly-long


: 1

**Create a Vault role so applications can access credentials**
Lease of 3 minutes. Renewable for 6 minutes.

In [12]:
vault write mdb1/roles/readonly-short \
    db_name=mysql \
    creation_statements="CREATE USER '{{name}}'@'%' IDENTIFIED BY '{{password}}';GRANT SELECT ON *.* TO '{{name}}'@'%';" \
    default_ttl="3m" \
    max_ttl="6m"

Success! Data written to: mdb1/roles/readonly-short


: 1

### Generate Dynamic Database Credentials
**Request Vault credentials by reading from the role**

In [13]:
vault read mdb1/creds/readonly-long

Key                Value
---                -----
lease_id           mdb1/creds/readonly-long/Y54cSkbg0Beg4eUxyO945mWd
lease_duration     1h
lease_renewable    true
password           0ixoR0gN-R9IZwXtrCCh
username           v-read-g0HgkQPU7


: 1

Now generate credentials for the role, "readonly-short", using a `curl` call against the Vault HTTP API and pipe the results to `jq` to make the JSON returned by the API easier to read:

In [14]:
curl -s -H "X-Vault-Token: root" "http://localhost:8200/v1/mdb1/creds/readonly-short" | jq | tee /tmp/readonly-short.creds

{[?2004l
  "request_id": "92d4255d-c60d-ac65-9ab6-401a5c33e29e",
  "lease_id": "mdb1/creds/readonly-short/PAIhQa3a7GYqFZcyjCiwR5zW",
  "renewable": true,
  "lease_duration": 180,
  "data": {
    "password": "lDqVTY9Vl-qJK-AQ657X",
    "username": "v-read-icVfUOIJp"
  },
  "wrap_info": null,
  "warnings": null,
  "auth": null
}


: 1

In these results, you see several thinggs:
* `lease_id` - used if you want to renew or revoke the credentials (as you will do in the next challenge).
* `username` and `password` - used to connect to the database.
* `renewable` - has the value `true`, indicating that the lifetime of the credentials can be extended using Vault's `sys/leases/renew` API endpoint.

Use the last set of credentials to connect to the local MySQL server with a command like this. Verify that you can see the databases on the MySQL server.

In [15]:
DB_USERNAME=$(jq -r .data.username /tmp/readonly-short.creds)
DB_PASSWORD=$(jq -r .data.password /tmp/readonly-short.creds)
docker exec -it mariadb mysql -u ${DB_USERNAME} -p${DB_PASSWORD} -e "show databases;"

+--------------------+4l
| Database           |
+--------------------+
| information_schema |
| mysql              |
| performance_schema |
+--------------------+


: 1

**Display database users** (note new users added by Vault)

In [90]:
docker exec -it mariadb mysql -u vaultadmin -p'vadmin' -e "select user from mysql.user;"

+------------------+
| User             |
+------------------+
| root             |
| v-read-J6TfRy4lE |
| v-read-lO8h5WAmk |
| vaultadmin       |
| mariadb.sys      |
+------------------+


: 1

**List the Vault leases for readonly-short role**

In [91]:
vault list sys/leases/lookup/mdb1/creds/readonly-short

Keys
----
mXWNfi8w8bG3cRgOk9y6PZTd


: 1

**Details on a lease** COPY LEASE ID ABOVE TO REPLACE END OF THIS LINE 

In [92]:
vault write sys/leases/lookup lease_id=mdb1/creds/readonly-short/mXWNfi8w8bG3cRgOk9y6PZTd

Key             Value
---             -----
expire_time     2021-05-27T00:49:06.862369488Z
id              mdb1/creds/readonly-short/mXWNfi8w8bG3cRgOk9y6PZTd
issue_time      2021-05-27T00:46:06.862369223Z
last_renewal    <nil>
renewable       true
ttl             1m55s


: 1

In [93]:
vault list sys/leases/lookup/mdb1/creds/readonly-short/

Keys
----
mXWNfi8w8bG3cRgOk9y6PZTd


: 1

### Renew and Revoke Database Credentials

In addition to using Vault's database secrets engine to generate credentials for databases, you can also use it to extend their lifetime or revoke them.

Generate new credentials against the readonly-short role.

In [94]:
vault read mdb1/creds/readonly-short

Key                Value
---                -----
lease_id           mdb1/creds/readonly-short/HrRi55Tos1ZZq27UFe1fuRCn
lease_duration     3m
lease_renewable    true
password           y0JgFewYYcG-HcM6TBZL
username           v-read-6scupkHX5


: 1

Renew lease with a command like this. We are setting the life of the credentials to 2 minutes.

In [96]:
vault write sys/leases/renew lease_id=mdb1/creds/readonly-short/HrRi55Tos1ZZq27UFe1fuRCn increment="120"

Key                Value
---                -----
lease_id           mdb1/creds/readonly-short/HrRi55Tos1ZZq27UFe1fuRCn
lease_duration     2m
lease_renewable    true


: 1

The `ttl` will tell you the remaining time to live of the lease and the credentials. When the lease expires, Vault will delete the credentials from MySQL.

Extending the lease will only work if the lease has not yet expired. Additionally, the lease on the credentials cannot be extended beyond the original time of their creation plus the duration given by the `max_ttl` parameter of the role.  If either of these conditions apply, you will get an error.

For instance, if you try to lookup a lease that has already expired, you will get an `invalid lease` error. If you try to extend the lease with an increment of 600 seconds (10 minutes), you will see an error like:

```
WARNING! The following warnings were returned from Vault:
    * TTL of "10m0s" exceeded the effective max_ttl of "2m17s";
    TTL value is capped accordingly
```

**Revoke a specific Vault lease**

In [97]:
vault lease revoke mdb1/creds/readonly-short/HrRi55Tos1ZZq27UFe1fuRCn

All revocation operations queued successfully!


: 1

**Revoke all Vault leases for a specific role**

In [98]:
vault lease revoke -prefix=true mdb1/creds/readonly-short

All revocation operations queued successfully!


: 1

In [ ]:
Congratulations on finishing the Vault Dynamic Database Credentials track.

### Cleanup

In [99]:
vault secrets disable mdb1

Success! Disabled the secrets engine (if it existed) at: mdb1/


: 1

**Shutdown Docker**

In [103]:
docker stop mariadb vault

Error response from daemon: No such container: mariadb
Error response from daemon: No such container: vault


: 1

**Remove the container**

In [102]:
docker rm mariadb vault

Error: No such container: mariadb
Error: No such container: vault


: 1

**END**